In [1]:

from vllm import LLM, SamplingParams
# Specify the maximum number of frames per video to be 4. This can be changed.
llm = LLM("/mnt/petrelfs/songmingyang/songmingyang/model/mm/Qwen2-VL-2B-Instruct", limit_mm_per_prompt={"image": 4})

/mnt/petrelfs/songmingyang/anaconda3/envs/r1-v/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 02-21 22:13:16 __init__.py:190] Automatically detected platform cuda.


2025-02-21 22:13:17,604	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 02-21 22:13:41 config.py:542] This model supports multiple tasks: {'embed', 'generate', 'score', 'reward', 'classify'}. Defaulting to 'generate'.
INFO 02-21 22:13:41 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='/mnt/petrelfs/songmingyang/songmingyang/model/mm/Qwen2-VL-2B-Instruct', speculative_config=None, tokenizer='/mnt/petrelfs/songmingyang/songmingyang/model/mm/Qwen2-VL-2B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collec

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.33s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.40it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.24it/s]



...

WARNING 02-21 22:14:02 model_runner.py:1288] Computed max_num_seqs (min(256, 32768 // 81920)) to be less than 1. Setting it to the minimum value of 1.


It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
Token indices sequence length is longer than the specified maximum sequence length for this model (81920 > 32768). Running this sequence through the model will result in indexing errors


WARNING 02-21 22:14:12 profiling.py:187] The context length (32768) of the model is too short to hold the multi-modal embeddings in the worst case (81920 tokens in total, out of which {'image': 65536, 'video': 16384} are reserved for multi-modal embeddings). This may cause certain multi-modal inputs to fail during inference, even when the input text is short. To avoid this, you should increase `max_model_len`, reduce `max_num_seqs`, and/or reduce `mm_counts`.
INFO 02-21 22:14:17 worker.py:267] Memory profiling takes 16.53 seconds
INFO 02-21 22:14:17 worker.py:267] the current vLLM instance can use total_gpu_memory (79.32GiB) x gpu_memory_utilization (0.90) = 71.39GiB
INFO 02-21 22:14:17 worker.py:267] model weights take 4.15GiB; non_torch_memory takes 0.21GiB; PyTorch activation peak memory takes 2.02GiB; the rest of the memory reserved for KV Cache is 65.00GiB.
INFO 02-21 22:14:17 executor_base.py:110] # CUDA blocks: 152146, # CPU blocks: 9362
INFO 02-21 22:14:17 executor_base.py:115]

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:17<00:00,  2.01it/s]

INFO 02-21 22:14:39 model_runner.py:1562] Graph capturing finished in 18 secs, took 1.90 GiB
INFO 02-21 22:14:39 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 38.64 seconds


In [19]:
from PIL import Image
import io
import base64,json
def process_jsonl(file_path):
    '''
        将jsonl文件转换为装有dict的列表
    '''
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            json_obj = json.loads(line)
            data.append(json_obj)
    return data

def write_jsonl(data, file_path):
    '''
        将list[dict]写入jsonl文件
    '''
    with open(file_path, 'w', encoding='utf-8') as file:
        for item in data:
            line = json.dumps(item,ensure_ascii=False)
            file.write(line + '\n')
def pil_to_base64(img: Image.Image, url_format = True) -> str:
    """
    Convert a PIL image to a base64 encoded string.
    
    Args:
        img (Image.Image): The PIL image to convert.
        
    Returns:
        str: Base64 encoded string representation of the image.
    """
    buffered = io.BytesIO()
    img.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')
    if url_format:
        img_str = f"data:image/jpeg;base64,{img_str}"
    return img_str

In [20]:
# Create the request payload.
messages = process_jsonl("/mnt/petrelfs/songmingyang/code/reasoning/R1-V/src/r1-v/local_scripts/scripts/notebooks/test.jsonl")
message = [{
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe this image in detail."},
    ],
},
# {
#     "role": "user",
#     "content": [
#         {"type": "text", "text": "Describe this image in detail."},
#     ],}
]
image_path = "/mnt/petrelfs/songmingyang/code/tools/test_imgs/roxy.jpeg"
image = Image.open(image_path).convert("RGB")
image_base64 = pil_to_base64(image)
new_image = {"type": "image_url", "image_url": {"url": f"{image_base64}"}}
message[0]["content"].append(new_image)



In [24]:
# Perform inference and log output.
outputs = llm.chat([message])

for o in outputs:
    generated_text = o.outputs[0].text
    print(generated_text)



Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.32it/s, est. speed input: 5532.75 toks/s, output: 53.36 toks/s]

The image is an illustration of an animated character. This character has a distinctive purple


In [29]:
outputs[0].outputs[0].token_ids

(785,
 2168,
 374,
 458,
 39034,
 315,
 458,
 11371,
 3668,
 13,
 1096,
 3668,
 702,
 264,
 34847,
 24932)